In [1]:
%load_ext autoreload
%autoreload 2

### Load model

In [ ]:
import torch
import torch.nn as nn
from torch.amp import autocast, GradScaler
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from tqdm.auto import tqdm
import datasets
import neptune
from loss_new import SGTLoss

import json

import hydra
from omegaconf import DictConfig, OmegaConf

from metrics import cos_metric, topk_intersection_metric, reconstruction_rank_metric

from collections import defaultdict

from dataset import SGTDataset

from sgt_model import SGTModel

In [2]:
device = torch.device('cpu')

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct")
tokenizer.pad_token = tokenizer.eos_token

llm = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct", device_map=device)

In [4]:
ds = load_dataset("ai2_arc", "ARC-Challenge")['train']

In [3]:
llm.model.embed_tokens.weight.shape

torch.Size([151936, 1536])

In [4]:
llm.model.embed_tokens

Embedding(151936, 1536)

### Test metrics

ranks

In [14]:
from test_metrics import test_reconstruction_ranks_zero


# test_reconstruction_ranks_zero("euclid")
test_reconstruction_ranks_zero("cosine")

RuntimeError: The size of tensor a (9) must match the size of tensor b (151936) at non-singleton dimension 0

nn_fr

In [5]:
from test_metrics import test_nn_fr_zero


test_nn_fr_zero()

In [6]:

from test_metrics import test_nn_fr_one_embed_wrong


test_nn_fr_one_embed_wrong()

In [7]:
from test_metrics import test_nn_fr_one_embed_wrong_with_attention_mask


test_nn_fr_one_embed_wrong_with_attention_mask()

mrp_fr

In [8]:
from test_metrics import test_mrp_fr_zero


test_mrp_fr_zero()

In [9]:
from test_metrics import test_mrp_fr_one_embed_wrong


test_mrp_fr_one_embed_wrong()

In [10]:
from test_metrics import test_mrp_fr_with_attention_mask


test_mrp_fr_with_attention_mask()

### DEBUG

In [11]:
import torch

from transformers import AutoModelForCausalLM, AutoTokenizer

from metrics import nn_fr_metric as nn_fr

device = torch.device('cpu')

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct")
tokenizer.pad_token = tokenizer.eos_token

llm = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct").to(device)

In [6]:
texts = ["The quick brown fox jumps over the lazy dog", "Happy pig"]
tokenized = tokenizer(texts, return_tensors='pt', padding=True).to(device)
tokenized

{'input_ids': tensor([[   785,   3974,  13876,  38835,  34208,    916,    279,  15678,   5562],
        [ 32847,  23694, 151645, 151645, 151645, 151645, 151645, 151645, 151645]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 0, 0, 0, 0, 0, 0, 0]])}

In [43]:
text = "The quick brown fox jumps over the lazy dog"
tokenized = tokenizer(text, return_tensors='pt').to(device)

clean_embeddings = llm.model.embed_tokens(tokenized['input_ids'])

print(clean_embeddings[0][1].norm())

clean_embeddings[0][1] += torch.randn_like(clean_embeddings[0][1])

print(clean_embeddings[0][1].norm())

# actual = nn_fr(clean_embeddings, llm.model.embed_tokens, tokenized['input_ids'], tokenized['attention_mask'])

tensor(0.9746, grad_fn=<LinalgVectorNormBackward0>)
tensor(38.5595, grad_fn=<LinalgVectorNormBackward0>)


tensor(100.)